# POC - Geração Automática de Atas de Reunião com IA

Este notebook implementa uma prova de conceito para geração automática de atas de reunião usando:
1. **Carregamento de áudio** - Suporte a arquivos .mp3, .wav, .m4a
2. **Diarização** - Separação de speakers com pyannote.audio  
3. **Transcrição** - Conversão de áudio para texto com Whisper
4. **Geração de ata** - Processamento com OpenAI API para criar ata estruturada

---

In [1]:
# Instalação das dependências necessárias
!pip install git+https://github.com/openai/whisper.git -q
!pip install openai -q
!pip install pyannote.audio -q
!pip install torch torchvision torchaudio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import whisper
import os
from openai import OpenAI
from pyannote.audio import Pipeline
import torch
from datetime import datetime
import json

## Configuração dos Modelos

In [4]:
from google.colab import userdata
# Configurar a API da OpenAI
# Substitua pela sua chave de API ou use variável de ambiente
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")  # ou os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Carregar modelo Whisper
print("Carregando modelo Whisper...")
whisper_model = whisper.load_model("small")  # Pode usar "medium" ou "large" para melhor qualidade
print("Modelo Whisper carregado com sucesso!")

Carregando modelo Whisper...


100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 52.5MiB/s]


Modelo Whisper carregado com sucesso!


In [5]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) N
Token is valid (permission: fineGrained).
The token `tcc-token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `tcc-toke

In [7]:
# Configurar pipeline de diarização
print("Configurando pipeline de diarização...")
# Nota: Para usar pyannote, você precisa aceitar os termos em: https://huggingface.co/pyannote/speaker-diarization e https://huggingface.co/pyannote/segmentation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1")
diarization_pipeline.to(device) # Mover o pipeline para o dispositivo (GPU ou CPU)
print("Pipeline de diarização configurado!")

Configurando pipeline de diarização...
Usando dispositivo: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

...

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/classifier.ckpt' -> '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speechbrain/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/label_encoder.txt' -> '/root/.cache/torch/pyannote/speechbrain/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /root/.cache/torch/pyannote/speechbrain/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /root/.cac

Pipeline de diarização configurado!


## Funções Principais

In [8]:
def transcribe_audio(audio_path):
    """
    Transcreve áudio usando Whisper
    """
    try:
        print(f"Transcrevendo áudio: {audio_path}")
        result = whisper_model.transcribe(audio_path, language="pt")
        return result["text"]
    except Exception as e:
        print(f"Erro na transcrição: {e}")
        return ""

def perform_diarization(audio_path):
    """
    Realiza diarização (separação de speakers) do áudio
    """
    try:
        print(f"Realizando diarização: {audio_path}")
        diarization = diarization_pipeline(audio_path)

        # Converter resultado para formato mais legível
        speakers_info = []
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            speakers_info.append({
                "speaker": speaker,
                "start": turn.start,
                "end": turn.end,
                "duration": turn.end - turn.start
            })

        return speakers_info
    except Exception as e:
        print(f"Erro na diarização: {e}")
        return []

## Geração de Ata com OpenAI

In [10]:
def generate_meeting_minutes(transcription, speakers_info=None):
    """
    Gera ata de reunião usando OpenAI API
    """
    try:
        # Preparar informações de speakers se disponível
        speaker_context = ""
        if speakers_info:
            unique_speakers = list(set([s["speaker"] for s in speakers_info]))
            speaker_context = f"\n\nParticipantes identificados: {', '.join(unique_speakers)}"

        system_prompt = """Você é um assistente especializado em gerar atas de reunião.
        Sua tarefa é analisar a transcrição fornecida e criar uma ata estruturada e professional.

        A ata deve conter:
        1. Cabeçalho com data e participantes
        2. Resumo executivo dos principais pontos
        3. Tópicos discutidos organizados por assunto
        4. Decisões tomadas e responsáveis
        5. Próximos passos e prazos
        6. Observações adicionais se necessário

        Mantenha um tom formal e objetivo. Organize as informações de forma clara e hierárquica."""

        user_prompt = f"""Transcrição da reunião:
        {transcription}
        {speaker_context}

        Por favor, gere uma ata completa e bem estruturada baseada nesta transcrição."""

        response = client.chat.completions.create(
            model="gpt-4o-mini",  # ou "gpt-3.5-turbo" para economia
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # Baixa criatividade para manter precisão
            max_tokens=2000
        )

        return response.choices[0].message.content

    except Exception as e:
        print(f"Erro na geração da ata: {e}")
        return ""

In [11]:
def process_meeting_audio(audio_path):
    """
    Função principal que processa o áudio completo:
    1. Carregamento do áudio
    2. Diarização
    3. Transcrição
    4. Geração da ata
    """
    print(f"=== Processando reunião: {audio_path} ===")
    print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Verificar se arquivo existe
    if not os.path.exists(audio_path):
        print(f"Erro: Arquivo não encontrado - {audio_path}")
        return None

    # Etapa 1: Diarização
    print("\n1. Realizando diarização...")
    speakers_info = perform_diarization(audio_path)
    print(f"   Encontrados {len(set([s['speaker'] for s in speakers_info]))} speakers diferentes")

    # Etapa 2: Transcrição
    print("\n2. Transcrevendo áudio...")
    transcription = transcribe_audio(audio_path)
    print(f"   Transcrição concluída: {len(transcription)} caracteres")

    # Etapa 3: Geração da ata
    print("\n3. Gerando ata de reunião...")
    meeting_minutes = generate_meeting_minutes(transcription, speakers_info)

    # Resultados
    results = {
        "audio_file": audio_path,
        "processing_date": datetime.now().isoformat(),
        "speakers_info": speakers_info,
        "transcription": transcription,
        "meeting_minutes": meeting_minutes
    }

    print(f"\n=== Processamento concluído em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===")
    return results

## Teste da POC

In [12]:
# Teste com um arquivo de áudio das reuniões CONEPE/CONSU
# Ajuste o caminho conforme necessário
# audio_file = "../data/raw/audio/conepe/2024-01-22_conepe_#52.wav"
audio_file = "entrevista_ufs_fm.mp3"

# Verificar se arquivo existe antes de processar
if os.path.exists(audio_file):
    print(f"Processando arquivo: {audio_file}")
    results = process_meeting_audio(audio_file)

    if results:
        print("\n" + "="*80)
        print("RESULTADOS DA POC")
        print("="*80)

        print(f"\nArquivo processado: {results['audio_file']}")
        print(f"Data de processamento: {results['processing_date']}")

        print(f"\nSpeakers identificados: {len(set([s['speaker'] for s in results['speakers_info']]))}")
        for speaker in set([s['speaker'] for s in results['speakers_info']]):
            total_time = sum([s['duration'] for s in results['speakers_info'] if s['speaker'] == speaker])
            print(f"  - {speaker}: {total_time:.1f}s")

        print(f"\nTranscrição ({len(results['transcription'])} caracteres):")
        print("-" * 50)
        print(results['transcription'][:500] + "..." if len(results['transcription']) > 500 else results['transcription'])

        print(f"\nAta de Reunião:")
        print("-" * 50)
        print(results['meeting_minutes'])

        # Salvar resultados
        output_file = f"../data/atas-geradas/ata_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"\nResultados salvos em: {output_file}")

else:
    print(f"Arquivo não encontrado: {audio_file}")
    print("Arquivos disponíveis:")
    audio_dir = "../data/raw/audio"
    if os.path.exists(audio_dir):
        for root, dirs, files in os.walk(audio_dir):
            for file in files:
                if file.endswith(('.wav', '.mp3', '.m4a')):
                    print(f"  - {os.path.join(root, file)}")
    else:
        print("Diretório de áudio não encontrado!")

Processando arquivo: entrevista_ufs_fm.mp3
=== Processando reunião: entrevista_ufs_fm.mp3 ===
Iniciado em: 2025-08-06 17:39:44

1. Realizando diarização...
Realizando diarização: entrevista_ufs_fm.mp3


/usr/local/lib/python3.11/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(


   Encontrados 2 speakers diferentes

2. Transcrevendo áudio...
Transcrevendo áudio: entrevista_ufs_fm.mp3
   Transcrição concluída: 21069 caracteres

3. Gerando ata de reunião...

=== Processamento concluído em: 2025-08-06 17:44:58 ===

RESULTADOS DA POC

Arquivo processado: entrevista_ufs_fm.mp3
Data de processamento: 2025-08-06T17:44:58.821718

Speakers identificados: 2
  - SPEAKER_01: 1225.8s
  - SPEAKER_00: 100.9s

Transcrição (21069 caracteres):
--------------------------------------------------
 A Universidade Federal de Sejip aprovou recentemente a criação do curso de graduação em inteligência artificial. Quem vai falar sobre o assunto a partir de agora com a gente aqui na UFIS FM é o professor Hendrik Macedo do Departamento de Computação da UFIS. Na satisfação ouvir o professor aqui na UFIS FM como surgiu a proposta de criação desse curso de bacharelado em IA. Boa tarde. Boa tarde, Josaphá. Eu e o que agradeço a oportunidade de esclarecer esse assunto. Veja bem, nós temos asso

In [13]:
results

{'audio_file': 'entrevista_ufs_fm.mp3',
 'processing_date': '2025-08-06T17:44:58.821718',
 'speakers_info': [{'speaker': 'SPEAKER_01',
   'start': 2.3597187500000003,
   'end': 3.7772187500000003,
   'duration': 1.4175},
  {'speaker': 'SPEAKER_00',
   'start': 7.202843750000001,
   'end': 31.722218750000003,
   'duration': 24.519375000000004},
  {'speaker': 'SPEAKER_01',
   'start': 32.04284375,
   'end': 154.09971875000002,
   'duration': 122.05687500000002},
  {'speaker': 'SPEAKER_00',
   'start': 154.35284375,
   'end': 162.45284375,
   'duration': 8.099999999999994},
  {'speaker': 'SPEAKER_01',
   'start': 162.57096875000002,
   'end': 280.99971875,
   'duration': 118.42874999999998},
  {'speaker': 'SPEAKER_00',
   'start': 281.32034375,
   'end': 298.48221875,
   'duration': 17.16187500000001},
  {'speaker': 'SPEAKER_01',
   'start': 298.48221875,
   'end': 566.7609687500001,
   'duration': 268.27875000000006},
  {'speaker': 'SPEAKER_00',
   'start': 536.0147187499999,
   'end': 5

## Notas e Próximos Passos

### Configurações Necessárias

1. **Chave da API OpenAI**: Configure sua chave da API OpenAI na célula de configuração
2. **Hugging Face Token**: Para usar pyannote.audio, você precisa:
   - Criar conta no Hugging Face
   - Aceitar os termos em: https://huggingface.co/pyannote/speaker-diarization
   - Configurar token de acesso

### Melhorias Possíveis

1. **Interface Gradio**: Adicionar interface web para upload de arquivos
2. **Modelos maiores**: Usar Whisper "medium" ou "large" para melhor qualidade
3. **Pós-processamento**: Adicionar correção ortográfica e formatação
4. **Templates**: Criar templates específicos para diferentes tipos de reunião
5. **Exportação**: Gerar PDFs e documentos Word da ata

### Custos Estimados

- **OpenAI API**: ~$0.03-0.06 por minuto de áudio (dependendo do modelo)
- **Processamento local**: Whisper e diarização rodam localmente (gratuito)

In [ ]:
# Função utilitária para listar e testar diferentes arquivos
def list_available_audio_files():
    """Lista todos os arquivos de áudio disponíveis"""
    audio_files = []
    audio_dir = "../data/raw/audio"

    if os.path.exists(audio_dir):
        for root, dirs, files in os.walk(audio_dir):
            for file in files:
                if file.endswith(('.wav', '.mp3', '.m4a')):
                    full_path = os.path.join(root, file)
                    rel_path = os.path.relpath(full_path, "..")
                    audio_files.append(rel_path)

    return sorted(audio_files)

def quick_test(audio_file_path):
    """Teste rápido com apenas transcrição (sem diarização para economizar tempo)"""
    print(f"Teste rápido: {audio_file_path}")

    if not os.path.exists(audio_file_path):
        print(f"Arquivo não encontrado: {audio_file_path}")
        return None

    # Apenas transcrição
    transcription = transcribe_audio(audio_file_path)

    # Gerar ata sem informação de speakers
    meeting_minutes = generate_meeting_minutes(transcription)

    print(f"\nTranscrição ({len(transcription)} chars):")
    print(transcription[:300] + "..." if len(transcription) > 300 else transcription)

    print(f"\nAta gerada:")
    print(meeting_minutes)

    return {"transcription": transcription, "meeting_minutes": meeting_minutes}

# Listar arquivos disponíveis
print("Arquivos de áudio disponíveis:")
available_files = list_available_audio_files()
for i, file in enumerate(available_files):
    print(f"{i+1:2d}. {file}")

# Exemplo de uso do teste rápido (descomente para usar)
# if available_files:
#     quick_test(available_files[0])

In [ ]:
# Desinstalar a versão atual do PyTorch, TorchVision e TorchAudio
!pip uninstall torch torchvision torchaudio -y

# Instalar uma versão específica do PyTorch compatível com o modelo pyannote
# Esta versão (1.13.1) foi escolhida por ser relativamente próxima da versão de treinamento
# do modelo (1.10.0) e por ter uma versão CUDA (11.7) compatível com as do Colab.
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

print("\nPor favor, REINICIE O AMBIENTE DE EXECUÇÃO (Runtime > Restart runtime) após a conclusão da instalação para que as alterações tenham efeito.")